# Fine-tuning MobileNetV2 on CIFAR-100
This notebook continues training a MobileNetV2-based model that has reached ~69.2% validation accuracy, with the goal of pushing it past 70%.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import matplotlib.pyplot as plt

2025-04-15 17:48:13.413984: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 17:48:13.458946: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 17:48:13.458988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 17:48:13.460835: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 17:48:13.470332: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 17:48:13.471050: I tensorflow/core/platform/cpu_feature_guard.cc:1

## Load previous best model and prepare dataset

In [2]:
# Load best model so far
model = load_model('checkpoints/mobilenetv2_cifar100_best.h5')

# Prepare data generators
IMG_SIZE = 96
BATCH_SIZE = 64

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

train_gen = train_datagen.flow(x_train, tf.keras.utils.to_categorical(y_train, 100), batch_size=BATCH_SIZE)
val_gen = ImageDataGenerator().flow(x_test, tf.keras.utils.to_categorical(y_test, 100), batch_size=BATCH_SIZE)

## Fine-tuning the last 40 layers of MobileNetV2

In [5]:
# Encontrar la capa MobileNetV2 dentro del modelo
base_model = None
for layer in model.layers:
    if isinstance(layer, tf.keras.Model) and layer.name.startswith('mobilenetv2'):
        base_model = layer
        break

# Asegurarse de que encontramos MobileNetV2
if base_model is not None:
    base_model.trainable = True
    # Congelar las primeras capas (ajustable)
    for layer in base_model.layers[:-40]:
        layer.trainable = False
else:
    raise ValueError("No se encontró MobileNetV2 en el modelo cargado.")

# Recompilar el modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


## Continue training with early stopping and learning rate scheduler

In [6]:
# Callbacks
checkpoint_cb = ModelCheckpoint('checkpoints/mobilenetv2_finetuned_best.h5', save_best_only=True)
early_stop_cb = EarlyStopping(patience=10, restore_best_weights=True)
lr_cb = ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-6)

# Train more epochs
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[checkpoint_cb, early_stop_cb, lr_cb],
    verbose=1
)

Epoch 1/30
782/782 [==============================] - ETA: 0s - loss: 3.7149 - accuracy: 0.1350

/usr/local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 52s 61ms/step - loss: 3.7149 - accuracy: 0.1350 - val_loss: 2.8900 - val_accuracy: 0.2573 - lr: 1.0000e-04
Epoch 2/30
782/782 [==============================] - 44s 57ms/step - loss: 3.1959 - accuracy: 0.2189 - val_loss: 2.6103 - val_accuracy: 0.3251 - lr: 1.0000e-04
Epoch 3/30
782/782 [==============================] - 42s 54ms/step - loss: 2.9748 - accuracy: 0.2643 - val_loss: 2.5272 - val_accuracy: 0.3553 - lr: 1.0000e-04
Epoch 4/30
782/782 [==============================] - 44s 56ms/step - loss: 2.8516 - accuracy: 0.2936 - val_loss: 2.4160 - val_accuracy: 0.3752 - lr: 1.0000e-04
Epoch 5/30
782/782 [==============================] - 43s 54ms/step - loss: 2.7663 - accuracy: 0.3107 - val_loss: 2.3096 - val_accuracy: 0.3943 - lr: 1.0000e-04
Epoch 6/30
782/782 [==============================] - 45s 57ms/step - loss: 2.6792 - accuracy: 0.3300 - val_loss: 2.2852 - val_accuracy: 0.4026 - lr: 1.0000e-04
Epoch 7/30
782/782 [=========================

## Final evaluation

In [7]:
model = load_model('checkpoints/mobilenetv2_finetuned_best.h5')
loss, acc = model.evaluate(val_gen)
print(f"Final Validation Loss: {loss:.4f}")
print(f"Final Validation Accuracy: {acc:.4f}")

157/157 [==============================] - 5s 27ms/step - loss: 1.9302 - accuracy: 0.5062
Final Validation Loss: 1.9302
Final Validation Accuracy: 0.5062
